In [12]:
import os
import sys; sys.path.append('../')
import yaml
import pickle
import datetime
from contextlib import suppress


import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.calibration import CalibrationDisplay


import torch
import torchmetrics
import torch.nn as nn
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter


import utils

In [13]:
with open('../configs/train.yaml', 'r') as stream:
    config = yaml.safe_load(stream)

In [14]:
v_size = config['split']['league']['val']
t_size = config['split']['league']['test']

In [15]:
train_df = pd.read_json(f'../parse/output/league/train_df.json')
print(len(train_df))

if v_size > 0:
    val_df = pd.read_json(f'../parse/output/league/val_df.json')
    print(len(val_df))
if t_size > 0:
    test_df = pd.read_json(f'../parse/output/league/test_df.json')
    print(len(test_df))

22299


In [16]:
train_df['part'] = 'train'
if v_size > 0: val_df['part'] = 'val'
if t_size > 0: test_df['part'] = 'test'

for eval_ in [True, ]:
    corpus = train_df.reset_index(drop=True)
    train_dataset = utils.nn.datasets.LeagueDataset(
        corpus=corpus, 
        evaluate_tokenize=eval_,
        indexes=corpus.index[(corpus['part'] == 'train')], 
        y_output='crossentropy',
    ).build()
    with open(f'output/train_cache{"_evaluated" if eval_ else ""}.pkl', 'wb') as p:
        pickle.dump(train_dataset.cache, p)

        
    if v_size > 0:
        corpus = pd.concat([corpus, val_df]).reset_index(drop=True)
        val_dataset = utils.nn.datasets.LeagueDataset(
            corpus=corpus, 
            evaluate_tokenize=eval_,
            indexes=corpus.index[(corpus['part'] == 'val')], 
            y_output='crossentropy',
        ).build()
        with open(f'output/val_cache{"_evaluated" if eval_ else ""}.pkl', 'wb') as p:
            pickle.dump(val_dataset.cache, p)
        
    if t_size > 0:
        corpus = pd.concat([corpus, test_df]).reset_index(drop=True)
        test_dataset = utils.nn.datasets.LeagueDataset(
            corpus=corpus, 
            evaluate_tokenize=eval_,
            indexes=corpus.index[(corpus['part'] == 'test')], 
            y_output='crossentropy',
        ).build()
        with open(f'output/test_cache{"_evaluated" if eval_ else ""}.pkl', 'wb') as p:
            pickle.dump(test_dataset.cache, p)

100%|████████████████████████████████████████████████████████████████████████████| 22299/22299 [04:51<00:00, 76.42it/s]


In [17]:
device = 'cuda'
batch_size = config['batch_size']['league']

In [18]:
train_dataset = utils.nn.datasets.LeagueDataset(
    corpus=None, 
    indexes=None, 
    y_output='crossentropy')
train_dataset.cache = pickle.load(open('output/train_cache_evaluated.pkl', 'rb'))
train_dataset.build()
train_loader = DataLoader(train_dataset, batch_size=config['batch_size']['league'], shuffle=False, num_workers=0)
print(len(train_dataset))

if v_size > 0:
    val_dataset = utils.nn.datasets.LeagueDataset(
        corpus=None, 
        indexes=None, 
        y_output='crossentropy')
    val_dataset.cache = pickle.load(open('output/val_cache_evaluated.pkl', 'rb'))
    val_dataset.build()
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size']['league'], shuffle=False, num_workers=0)
    print(len(val_dataset))
    
if t_size > 0:
    test_dataset = utils.nn.datasets.LeagueDataset(
        corpus=None, 
        indexes=None, 
        y_output='crossentropy')
    test_dataset.cache = pickle.load(open('output/test_cache_evaluated.pkl', 'rb'))
    test_dataset.build()
    test_loader = DataLoader(test_dataset, batch_size=config['batch_size']['league'], shuffle=False, num_workers=0)
    print(len(test_dataset))

14843


In [19]:
def get_writer(name:str):
    log_dir = f"logs/fit/prematch/{name}"
    writer = SummaryWriter(log_dir)
    return writer

def compute_metrics(y_pred, y_true):
    return {
        'Acc': float(torchmetrics.functional.accuracy(y_pred, y_true)),
        'AUC': float(torchmetrics.functional.auroc(y_pred, y_true)),
        'MAP': float(torchmetrics.functional.average_precision(y_pred, y_true)),
        'LogLoss': metrics.log_loss(y_true, y_pred),
    }

tokenizer = utils.tokenizer.Tokenizer(path="../parse/output/tokenizer_league.pkl")

utils.nn.PrematchModel(
    teams_num=max(tokenizer.teams_vocab.values()),
    regression=False,
).summary()

Layer (type:depth-idx)                   Param #
├─Embedding: 1-1                         968
├─WindowedGamesFeatureEncoder: 1-2       --
|    └─StatsEncoder: 2-1                 --
|    |    └─Linear: 3-1                  800
|    |    └─Linear: 3-2                  800
|    |    └─ModuleList: 3-3              960
|    └─ResultEncoder: 2-2                --
|    |    └─Embedding: 3-4               64
|    └─Embedding: 2-3                    1,056
├─RNN: 1-3                               --
|    └─GRU: 2-4                          25,344
├─OutputHead: 1-4                        --
|    └─Sequential: 2-5                   --
|    |    └─Linear: 3-5                  1,152
|    |    └─GELU: 3-6                    --
|    |    └─Dropout: 3-7                 --
|    |    └─Linear: 3-8                  768
|    |    └─GELU: 3-9                    --
|    |    └─Dropout: 3-10                --
|    └─Sequential: 2-6                   --
|    |    └─Linear: 3-11                 1,536
|    |   

In [37]:
preds = []
results = []
models_names = []

date = datetime.datetime.now().strftime("%Y.%m.%d - %H-%M")
for n in range(5):    
    torch.cuda.empty_cache()
    
    name = f'Ensemble {n} {date}'
    writer = get_writer(name)

    model = utils.nn.PrematchModel(
        teams_num=max(tokenizer.teams_vocab.values()),
        regression=False,
    )

    optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=0.2)
    scheduler = utils.nn.shedulers.TransformerLRScheduler(
        optimizer=optimizer,
        init_lr=1e-5,
        peak_lr=1e-3,
        final_lr=1e-5,
        final_lr_scale=0.01,
        warmup_steps=10,
        decay_steps=70,
    )

    trainer = utils.nn.trainers.PremtachTrainer(
        model=model, 
        loss_fn=torch.nn.CrossEntropyLoss(), 
        sheduler=scheduler,
        optimizer=optimizer,
        metric={
            "Acc": torchmetrics.Accuracy(num_classes=2),
            "AUC": torchmetrics.AUROC(num_classes=2),
            "MAP": torchmetrics.AveragePrecision(num_classes=2),
        },
        device=device
    )

    # --------------------------------------------------------- #
    wait = 0
    patience = 50

    epoch = 0
    best_loss = -np.inf

    with suppress(KeyboardInterrupt):
        while epoch < 80:
            # --------------------------------------------- #
            # Train
            train_loss = trainer.train_epoch(train_loader)
            writer.add_scalar('LogLoss/train', train_loss, epoch)
            writer.add_scalar('Acc/train', trainer.metric['Acc'].compute(), epoch)
            writer.add_scalar('AUC/train', trainer.metric['AUC'].compute(), epoch)
            writer.add_scalar('MAP/train', trainer.metric['MAP'].compute(), epoch)
            
            # --------------------------------------------- #
            # Val
            if v_size > 0:
                val_pred, val_true = trainer.predict(val_loader)
                val_pred = val_pred.softmax(dim=1)[:, 1]
                val_metrics = compute_metrics(val_pred, val_true)
                for _metric in val_metrics:
                    writer.add_scalar(f'{_metric}/val', val_metrics[_metric], epoch)
                
            # --------------------------------------------- #
            # Test    
            if t_size > 0:
                test_pred, test_true = trainer.predict(test_loader)
                test_pred = test_pred.softmax(dim=1)[:, 1]
                test_metrics = compute_metrics(test_pred, test_true)
                for _metric in test_metrics:
                    writer.add_scalar(f'{_metric}/test', test_metrics[_metric], epoch)
                
            # --------------------------------------------- #
            # EarlyStopping
            wait, epoch = wait+1, epoch+1
            # if val_metrics['AUC'] > best_loss:
                # torch.save(trainer.checkpoint(), f'output/models_w/prematch/{name}.torch')
                # best_loss = val_metrics['AUC']
                # wait = 0
        checkpoint = trainer.checkpoint()
        checkpoint['kwargs'] = {
            'teams_num': max(tokenizer.teams_vocab.values()), 
            'regression': False,
        }
        torch.save(checkpoint, f'output/models_w/prematch/{name}.torch')
        
    models_names.append(name)
    # --------------------------------------------- #
    # Load and evaluate
    if t_size > 0 or v_size > 0:
    
        checkpoint = torch.load(f'output/models_w/prematch/{name}.torch')
        trainer.model.load_state_dict(checkpoint['model'])

        test_pred, test_true = trainer.predict(test_loader if t_size > 0 else val_loader)
        test_pred = test_pred.softmax(dim=1)[:, 1]

        test_metrics = compute_metrics(test_pred, test_true)
        test_metrics['epoch'] = epoch

        preds.append(test_pred)
        results.append(test_metrics)
        
    
if t_size > 0 or v_size > 0:
    preds = torch.vstack(preds)

C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AUROC` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)
C:\Users\royta\AppData\Local\Programs\Python\Python310\lib\site-packages\torchmetrics\utilities\prints.py:36: UserWarning: Metric `AveragePrecision` will save all targets and predictions in buffer. For large datasets this may lead to large memory footprint.
  warnings.warn(*args, **kwargs)


In [38]:
models_names

['Ensemble 0 2022.09.14 - 21-59']

In [39]:
models_names = [
    'Ensemble 0 2022.09.14 - 21-46',
    'Ensemble 1 2022.09.14 - 21-46',
    'Ensemble 2 2022.09.14 - 21-46',
    'Ensemble 3 2022.09.14 - 21-46',
    'Ensemble 4 2022.09.14 - 21-46',
    'Ensemble 0 2022.09.14 - 21-59'
]

In [40]:
preds = []
results = []
for name in models_names:
    checkpoint = torch.load(f'output/models_w/prematch/{name}.torch')
    trainer.model.load_state_dict(checkpoint['model'])
        
    pred, true = trainer.predict(train_loader)
    pred = pred.softmax(dim=1)[:, 1]

    preds.append(pred)
    results.append(compute_metrics(pred, true))
    
preds = torch.vstack(preds)
ensemble_mean_pred = preds.mean(dim=0)

In [41]:
models_num = preds.shape[0]

In [42]:
(preds - ensemble_mean_pred).std(dim=1)

tensor([0.0459, 0.0465, 0.0450, 0.0456, 0.0475, 0.1120])

In [55]:
torch.vstack([(preds[n] - preds).std(dim=1) for n in range(models_num)]).mean(dim=0)

tensor([0.0658, 0.0663, 0.0648, 0.0655, 0.0668, 0.1169])

In [33]:
preds[0]

tensor([0.4993, 0.5597, 0.5439,  ..., 0.4772, 0.3861, 0.6793])

In [34]:
ensemble_mean_pred

tensor([0.5440, 0.5614, 0.5854,  ..., 0.4954, 0.3792, 0.6384])

In [23]:
compute_metrics(ensemble_mean_pred, true)

{'Acc': 0.6323519349098206,
 'AUC': 0.6847135424613953,
 'MAP': 0.6849225759506226,
 'LogLoss': 0.638383864683318}

In [10]:
preds = []
results = []
for name in models_names:
    checkpoint = torch.load(f'output/models_w/prematch/{name}.torch')
    trainer.model.load_state_dict(checkpoint['model'])
        
    pred, true = trainer.predict(test_loader)
    pred = pred.softmax(dim=1)[:, 1]

    preds.append(pred)
    results.append(compute_metrics(pred, true))
    
preds = torch.vstack(preds)
ensemble_mean_pred = preds.mean(dim=0)

In [11]:
compute_metrics(ensemble_mean_pred, true)

{'Acc': 0.5718358159065247,
 'AUC': 0.6068392992019653,
 'MAP': 0.6015220284461975,
 'LogLoss': 0.6759956551454495}

In [ ]:
X = preds.std(dim=0)[:, None].numpy()
Y = (ensemble_mean_pred.round() != true).numpy().astype('float32')

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(X, Y, random_state=69, train_size=0.9)

In [ ]:
xtrain.shape

(13137, 1)

In [ ]:
smote = SMOTE()
xtrain, ytrain = smote.fit_resample(xtrain, ytrain)

In [ ]:
xtrain.shape

(16548, 1)

In [207]:
clf = LogisticRegression().fit(xtrain, ytrain)
pred = clf.predict_proba(xtest)

compute_metrics(torch.from_numpy(pred[:, 1]), torch.from_numpy(ytest.astype('int32')))

{'Acc': 0.516438364982605,
 'AUC': 0.4909028708934784,
 'MAP': 0.35196858644485474,
 'LogLoss': 0.6933551910620996}

In [187]:
clf = DecisionTreeClassifier(random_state=0).fit(xtrain, ytrain)
pred = clf.predict_proba(xtest)

compute_metrics(torch.from_numpy(pred[:, 1]), torch.from_numpy(ytest.astype('int32')))

{'Acc': 0.5109589099884033,
 'AUC': 0.4926212430000305,
 'MAP': 0.3605821132659912,
 'LogLoss': 16.614337861865202}

In [11]:
batch_size = 512
stride = 32

to_plot = {
    'Acc': [],
    'AUC': [],
    'MAP': [],
    'LogLoss': [],
}
for batch in range(0, len(true)-batch_size, stride):
    _m = compute_metrics(ensemble_mean_pred[batch:batch+batch_size], true[batch:batch+batch_size])
    
    for key in _m: to_plot[key].append(_m[key])
    
del to_plot['LogLoss']

In [ ]:
for k in to_plot:
    plt.plot(to_plot[k])

plt.legend(to_plot.keys())

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(16, 8))

ax.set_title("Calibration curve")
display = CalibrationDisplay.from_predictions(
    true,
    ensemble_mean_pred, 
    n_bins=10,
    name="Ensembling models",
    ax=ax,
)


major_ticks = np.arange(0, 1+1e-6, 0.1)
minor_ticks = np.arange(0, 1+1e-6, 0.025)

ax.set_xticks(major_ticks)
ax.set_xticks(minor_ticks, minor=True)
ax.set_yticks(major_ticks)
ax.set_yticks(minor_ticks, minor=True)

ax.grid(which='minor', alpha=0.2)
ax.grid(which='major', alpha=0.5)


_ax = ax.twinx()
sns.distplot(ensemble_mean_pred, ax=_ax, color='orange', hist=True, kde=False, rug=True)

#

### Evaluate against book odds

In [15]:
odds = pd.read_csv("../odds.csv")

M = (1/odds['radiant_odd'] + 1/odds['dire_odd']) - 1

odds['r_pred'] = (1-M)/odds['radiant_odd']
odds['d_pred'] = (1-M)/odds['dire_odd']

# odds = odds[odds['match_id'] > 6000000000]

compute_metrics(
    y_pred=torch.from_numpy(1/odds['radiant_odd'].values), 
    y_true=torch.from_numpy(odds['radiant_win'].values.astype('int64'))
)

true = []
my_pred = []
book_pred = []

_, test_true = trainer.predict(test_loader)
for idx, batch in enumerate(DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)):
    # if idx == 200: break
    match_id = batch['match_id'].item()

    if (odds['match_id'] == match_id).any():
        true.append( test_true[idx].item() )
        my_pred.append( ensemble_mean_pred[idx].item() )
        book_pred.append( odds[odds['match_id'] == match_id]['r_pred'].values[0] )
        
true = torch.IntTensor(true)
my_pred = torch.Tensor(my_pred)
book_pred = torch.Tensor(book_pred)

print("My metrics")
print(compute_metrics(my_pred, true))

print("Book metrics")
print(compute_metrics(book_pred, true))

My metrics
{'Acc': 0.5856443643569946, 'AUC': 0.6177918314933777, 'MAP': 0.6079134345054626, 'LogLoss': 0.6711387628443284}
Book metrics
{'Acc': 0.5481239557266235, 'AUC': 0.5502524375915527, 'MAP': 0.5543672442436218, 'LogLoss': 0.6985431941648487}


In [16]:
model.eval()
for idx, batch in enumerate(DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=0)):
    break

batch = utils.nn.tools.batch_to_device(batch, 'cuda')
with torch.no_grad():
    out = model(batch)
    print(out.softmax(dim=1)[:, 1].cpu())
    
    
    batch['d_window'], batch['r_window'] = batch['r_window'], batch['d_window']
    batch['teams']['radiant'], batch['teams']['dire'] = batch['teams']['dire'], batch['teams']['radiant']

    out = model(batch)
    print(out.softmax(dim=1)[:, 0].cpu())

tensor([0.5027, 0.3681, 0.5266, 0.3381, 0.4116, 0.4513, 0.4528, 0.4322])
tensor([0.4849, 0.3665, 0.5477, 0.3206, 0.4370, 0.4182, 0.4178, 0.3975])


In [130]:
# odds = pd.concat([pd.read_json("../scarpe/output/odds1.json"), pd.read_json("../scarpe/output/odds2.json")])
# odds = pd.read_json("../scarpe/output/odds1.json")
# odds = odds.drop_duplicates('match_id').reset_index()

# true = []
# my_pred = []
# book_pred = []

# _, test_true = trainer.predict(test_loader)
# for idx, batch in enumerate(DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)):
#     # if idx == 200: break
#     match_id = batch['match_id'].item()

#     if (odds['match_id'] == match_id).any():
#         true.append( test_true[idx].item() )
#         my_pred.append( ensemble_mean_pred[idx].item() )
#         book_pred.append( 1/odds[odds['match_id'] == match_id]['r_odd'].values[0] )
        
# true = torch.IntTensor(true)
# my_pred = torch.Tensor(my_pred)
# book_pred = torch.Tensor(book_pred)

# print("My metrics")
# print(compute_metrics(my_pred, true))

# print("Book metrics")
# print(compute_metrics(book_pred, true))

#

### Debug

In [226]:
team1 = tokenizer.teams_vocab[7119388]
team2 = tokenizer.teams_vocab[15]

batches = []
for batch in train_loader:
    b = ((batch['teams']['dire'] == team1) & (batch['teams']['radiant'] == team2)) | ((batch['teams']['dire'] == team2) & (batch['teams']['radiant'] == team1))
    if b.any():
        batches.append([batch, b])

In [227]:
team1, team2

(23, 5)

In [230]:
batch = batches[-1]
batch, b = batch
model.eval()
output = model(utils.nn.tools.batch_to_device(batch, 'cuda'))

In [232]:
batch['match_id'][b]

tensor([[6676393091],
        [6676488286],
        [6705859209],
        [6705943008],
        [6707542480],
        [6707633683],
        [6707714718],
        [6707754788]], device='cuda:0')

In [233]:
batch['teams']['radiant'][b]

tensor([ 5, 23,  5,  5,  5, 23, 23, 23], device='cuda:0')

In [234]:
batch['teams']['dire'][b]

tensor([23,  5, 23, 23, 23,  5,  5,  5], device='cuda:0')

In [235]:
batch['y'][b]

tensor([1, 0, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [239]:
output[b].softmax(dim=1)[:, 1]

tensor([0.7575, 0.2747, 0.5085, 0.6143, 0.5651, 0.4083, 0.4610, 0.4550],
       device='cuda:0', grad_fn=<SelectBackward0>)

In [237]:
batch.keys()

dict_keys(['match_id', 'r_window', 'd_window', 'y', 'y_r_stats', 'y_d_stats', 'teams'])

In [238]:
train_df[train_df['match_id'] == 6705859209][[f"{s}_account_id" for s in utils.base.ConfigBase.RADIANT_SIDE]]

,0_account_id,1_account_id,2_account_id,3_account_id,4_account_id
22019,898754153,173978074,118134220,157475523,111114687
